## refer to: https://www.kaggle.com/raddar/paris-madness

# import libraries

In [1]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from scipy.interpolate import UnivariateSpline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import collections
import datetime
import copy
import mlflow

pd.set_option("display.max_column", 999)
# print(os.listdir("../input"))

# setting mlflow

# Load the data!

In [17]:
path = '../W/input/google-cloud-ncaa-march-madness-2020-division-1-womens-tournament/WDataFiles_Stage1/'
submit_csv_path = '../W/input/google-cloud-ncaa-march-madness-2020-division-1-womens-tournament'
output_path = '../output/W/'

tourney_results = pd.read_csv(os.path.join(path, 'WNCAATourneyDetailedResults.csv'))
seeds = pd.read_csv(os.path.join(path, 'WNCAATourneySeeds.csv'))
regular_results = pd.read_csv(os.path.join(path, 'WRegularSeasonDetailedResults.csv'))
cities = pd.read_csv(os.path.join(path, 'Cities.csv'))
conferences = pd.read_csv(os.path.join(path, 'Conferences.csv'))
wGameCities = pd.read_csv(os.path.join(path, 'WGameCities.csv'))
wNCAATourneySlots = pd.read_csv(os.path.join(path, 'WNCAATourneySlots.csv'))
wSeasons = pd.read_csv(os.path.join(path, 'WSeasons.csv'))
wTeamConferences = pd.read_csv(os.path.join(path, 'WTeamConferences.csv'))
wTeams = pd.read_csv(os.path.join(path, 'WTeams.csv'))
wPlayers = pd.read_csv(os.path.join(submit_csv_path, 'WPlayers.csv'))


In [4]:
# 都市については、regionとの距離とか出したいけど難しそうなので後回し
cities.head()

,CityID,City,State
0,4001,Abilene,TX
1,4002,Akron,OH
2,4003,Albany,NY
3,4004,Albuquerque,NM
4,4005,Allentown,PA


In [6]:
# あまり使わなそう
conferences.head()

,ConfAbbrev,Description
0,a_sun,Atlantic Sun Conference
1,a_ten,Atlantic 10 Conference
2,aac,American Athletic Conference
3,acc,Atlantic Coast Conference
4,aec,America East Conference


In [8]:
# 都市の利用法思いついたら
wGameCities.head()

,Season,DayNum,WTeamID,LTeamID,CRType,CityID
0,2010,11,3103,3237,Regular,4002
1,2010,11,3104,3399,Regular,4085
2,2010,11,3110,3224,Regular,4363
3,2010,11,3111,3267,Regular,4158
4,2010,11,3119,3447,Regular,4367


In [10]:
# slot自体がよくわからん
wNCAATourneySlots.head()

,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15
2,R1W3,W03,W14
3,R1W4,W04,W13
4,R1W5,W05,W12


In [12]:
# あまり意味なさそう
wSeasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1998,10/27/1997,East,Midwest,Mideast,West
1,1999,10/26/1998,East,Mideast,Midwest,West
2,2000,11/1/1999,East,Midwest,Mideast,West
3,2001,10/30/2000,East,Midwest,Mideast,West
4,2002,10/29/2001,East,West,Mideast,Midwest


In [14]:
# これは使えそう。因縁のcsv
wTeamConferences.head()

,Season,TeamID,ConfAbbrev
0,1998,3102,wac
1,1998,3103,mac
2,1998,3104,sec
3,1998,3106,swac
4,1998,3108,swac


In [16]:
# 使わなそう
wTeams.head()

,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M


In [9]:
sub = pd.read_csv(os.path.join(submit_csv_path, 'WSampleSubmissionStage1_2020.csv'))

def transform_test(test):
    # sub["Season"] = 2018 # なぜ2018年固定？
    test['Season'] = test['ID'].apply(lambda x: x[0:4]).astype(int)
    test["T1_TeamID"] = test["ID"].apply(lambda x: x[5:9]).astype(int)
    test["T2_TeamID"] = test["ID"].apply(lambda x: x[10:14]).astype(int)
    return test
sub = transform_test(sub)
sub.head()

,ID,Pred,Season,T1_TeamID,T2_TeamID
0,2015_3106_3107,0.5,2015,3106,3107
1,2015_3106_3110,0.5,2015,3106,3110
2,2015_3106_3113,0.5,2015,3106,3113
3,2015_3106_3114,0.5,2015,3106,3114
4,2015_3106_3116,0.5,2015,3106,3116


In [19]:
wPlayers.head()

,PlayerID,LastName,FirstName,TeamID
0,1,Adams,Alyssa,3101
1,2,Allen,Sierra,3101
2,3,Avery,Ellyn,3101
3,4,Cooper,Jenna,3101
4,5,Dimba,Lizzy,3101


# Data preparing!

In [10]:
class Preprocessor_NCAA:
    SWAP_COLS = ['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
                                'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
                                'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']

    STATS_EXC_COLS = ['DayNum', 'NumOT', 'T2_TeamID', 'location']
    
    def __init__(self, regular_results, tourney_results, seeds, train_flag=True):
        self.regular_data = regular_results
        self.tourney_data = tourney_results
        self.seeds = seeds
        self.train_flag = train_flag
        
    
    def fit(self):
        swap_regular_data = self.swap_data(self.regular_data)
#         return swap_regular_data
        encoded_regular_df = self.encode_data(swap_regular_data)
        self.regular_data = self.feature_eng_data(encoded_regular_df)
        
        if self.train_flag:
            self.tourney_data = self.swap_data(self.tourney_data)[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score']]

        self.cols =  list(set(self.regular_data.columns) - set(self.STATS_EXC_COLS))
        location_df_list = self.get_data_byLocation(self.regular_data)
        last14days_stats = self.get_lastNdays_data(self.regular_data, 14) # 14日前までのデータ同様、ホームだけ、アウェーだけ、ニュートラルだけのデータ作ってjoin_mean
        
        train_df = self.join_MeanValue(self.regular_data[self.cols], self.tourney_data)
        
        location_df_list.append(last14days_stats) # 絶対もっと綺麗にかける
        for df in location_df_list:
            train_df = self.join_MeanValue(df, train_df)
        train_df = self.join_GLM(train_df)
        train_df = self.join_seed(train_df)
        
        return train_df
    
    def swap_data(self, df):
        copy_df = df.copy()
        dfswap = copy_df[self.SWAP_COLS]
        dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
        dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
#         copy_df.columns.values[6] = 'locationXX'
#         dfswap.columns.values[6] = 'locationXX'    

        copy_df.columns = [x.replace('W','T1_').replace('L','T2_') if x!='WLoc' else 'location' for x in list(copy_df.columns)]
        dfswap.columns = [x.replace('L','T1_').replace('W','T2_') if x!='WLoc' else 'location' for x in list(dfswap.columns)]

        return pd.concat([copy_df, dfswap]).reset_index(drop=True)

    def encode_data(self, df):
        df.loc[df.location=='N','location'] = '0'
        df.loc[df.location=='H','location'] = '1'
        df.loc[df.location=='A','location'] = '-1'
        df.location = df.location.astype(int)
        
        return df
    
    def feature_eng_data(self, df):
        df['PointDiff'] = df['T1_Score'] - df['T2_Score']
        # 追加
        df['T1_win'] = np.where(df['PointDiff']>0, 1, 0) # 勝ち負け
        df['T2_win'] = np.where(df['PointDiff']>0, 0, 1)
        for t in ['T1', 'T2']:
            df[f'{t}_FGmissed'] = df[f'{t}_FGA'] - df[f'{t}_FGM'] # シュートミス
            df[f'{t}_FGmissed3'] = df[f'{t}_FGA3'] - df[f'{t}_FGM3'] # シュートミス
            df[f'{t}_FTmissed'] = df[f'{t}_FTA'] - df[f'{t}_FTM'] # フリースローミス
            
            df[f'{t}_FGsuccess_ratio'] = df[f'{t}_FGM'] / df[f'{t}_FGA'] # シュート成功率
            df[f'{t}_FG3success_ratio'] = df[f'{t}_FGM3'] / df[f'{t}_FGA3'] # 3Pシュート成功率
            df[f'{t}_FTsuccess_ratio'] = df[f'{t}_FTM'] / df[f'{t}_FTA'] # フリースロー成功率
            
            df[f'{t}_FG2ratio'] = df[f'{t}_FGM'] / df[f'{t}_Score'] # 2点シュートの割合
            df[f'{t}_FG3ratio'] = df[f'{t}_FGM3'] / df[f'{t}_Score'] # 3Pシュートの割合
            df[f'{t}_FGratio'] = df[f'{t}_FGM'] + df[f'{t}_FGM3'] / df[f'{t}_Score'] # FGの割合
            df[f'{t}_FTratio'] = df[f'{t}_FTM'] / df[f'{t}_Score'] # フリースローの割合
            
            df[f'{t}_Rratio'] = df[f'{t}_DR'] / df['T1_OR'] + df['T1_DR'] + df['T2_OR'] + df['T2_DR'] # リバウンド率
            df[f'{t}_ORratio'] = df[f'{t}_OR'] / df[f'{t}_OR'] + df[f'{t}_DR']
            df[f'{t}_DRratio'] = df[f'{t}_DR'] / df[f'{t}_OR'] + df[f'{t}_DR']
            
            df[f'{t}_ASTratio'] = df[f'{t}_Ast'] / df[f'{t}_FGM'] # アシスト率
            df[f'{t}_PF_byPeriod'] = df[f'{t}_PF'] / 4 # ピリオドごとの平均、PFってこんなに多いの！？
            df['T1_over5PF'] = np.where(df[f'{t}_PF_byPeriod']>=5, 1, 0) # 平均が5ファウル超えているか(チームファウル)

        df['T1_ORratio_op']  = df['T1_OR'] / df['T1_OR'] + df['T2_DR'] # 攻撃時のリバウンド、守備時のリバウンド(リストをスライスする形ならfor文に組み込めた)
        df['T1_DRratio_op']  = df['T1_DR'] / df['T1_DR'] + df['T2_OR']
        df['T2_ORratio_op']  = df['T2_OR'] / df['T1_DR'] + df['T2_OR']
        df['T2_DRratio_op']  = df['T2_DR'] / df['T1_OR'] + df['T2_DR']

        df['T1_FGblock'] = df['T1_Blk'] / df['T2_FGmissed'] # ブロック率
        df['T2_FGblock'] = df['T2_Blk'] / df['T1_FGmissed']
        df['T1_FGAblock'] = df['T1_Blk'] / df['T2_FGA'] # ブロック率２ どっちがいいかわからないから両方作った 
        df['T2_FGAblock'] = df['T2_Blk'] / df['T1_FGA']
        
        return df

    def join_MeanValue(self, df, toJoin_df):
        season_statistics = self.get_mean_value(df)
        season_statistics_T1 = season_statistics.copy()
        season_statistics_T2 = season_statistics.copy()

        season_statistics_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") if not x == 'Season' else x for x in list(season_statistics_T1.columns)]
        season_statistics_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") if not x == 'Season' else x for x in list(season_statistics_T2.columns)]
        
        toJoin_df = pd.merge(toJoin_df, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
        toJoin_df = pd.merge(toJoin_df, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')
        
        return toJoin_df

    def get_mean_value(self, df):
        keys = ["Season", 'T1_TeamID']
        funcs = [np.mean]
        
        season_statistics = df.groupby(keys).agg(funcs).reset_index()
        season_statistics.columns = [''.join(col).strip() for col in season_statistics.columns.values]
        
        return season_statistics
    
    def get_data_byLocation(self, df):
        home_df = df.loc[df.location ==1].reset_index(drop=True)[self.cols]
        home_df = self.column_addNamed(home_df, '_location{}'.format('1'))
        
        not_home_df = df.loc[df.location !=1].reset_index(drop=True)[self.cols]
        not_home_df = self.column_addNamed(not_home_df, '_location{}'.format('_not1'))
        return [home_df, not_home_df]
            
    
    def get_lastNdays_data(self, df, n):
        # 追加：ここはwinかどうか以外も足していい気がする
        lastNdays_stats = df.loc[df.DayNum>=(132-n)].reset_index(drop=True)[self.cols] # Regular: 0~132
        lastNdays_stats = self.column_addNamed(lastNdays_stats, '_last{}days'.format(n))
        return lastNdays_stats
    
    def column_addNamed(self, df, addName):
        not_add_cols = ['T1_TeamID', 'Season']
        df.columns = [col + addName if  col not in tuple(not_add_cols) else col for col in df.columns]
        return df
    
    def join_GLM(self, toJoin_df):
        glm_quality = self.fit_predict_GLM()
        glm_quality_T1 = glm_quality.copy()
        glm_quality_T2 = glm_quality.copy()
        glm_quality_T1.columns = ['T1_TeamID','T1_quality','Season']
        glm_quality_T2.columns = ['T2_TeamID','T2_quality','Season']
        
        toJoin_df = pd.merge(toJoin_df, glm_quality_T1, on = ['Season', 'T1_TeamID'], how = 'left')
        toJoin_df = pd.merge(toJoin_df, glm_quality_T2, on = ['Season', 'T2_TeamID'], how = 'left')
        
        return toJoin_df
        
    def fit_predict_GLM(self):
        self.make_GLMtrain()
        glm_quality = pd.concat([self.team_quality(season) for season in  range(2010, 2020)]).reset_index(drop=True)
        return glm_quality
    
    def make_GLMtrain(self):
        regular_season_effects = self.regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff', 'T1_win']].copy()
        regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
        regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
        march_madness = pd.merge(self.seeds[['Season','TeamID']],self.seeds[['Season','TeamID']],on='Season')
        march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
        march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
        march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)
        # シーズンごとのシード内の組み合わせ全てを作成、レギュラーシーズン中に行われなかった試合についてはteam_qualityがnullになる（joinで外れる）
        regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])
        self.regular_season_effects = regular_season_effects
    
    def team_quality(self, season):
        formula = 'T1_win~T1_TeamID+T2_TeamID-1'
        glm = sm.GLM.from_formula(formula=formula, 
                                  data=self.regular_season_effects.loc[self.regular_season_effects.Season==season,:], 
                                  family=sm.families.Binomial()).fit()

        quality = pd.DataFrame(glm.params).reset_index()
        quality.columns = ['TeamID','quality']
        quality['Season'] = season
        quality['quality'] = np.exp(quality['quality'])
        quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
        quality['TeamID'] = quality['TeamID'].apply(lambda x: x[-5:-1]).astype(int)
        return quality
    
    def join_seed(self, toJoin_df):
        seeds['seed'] = self.seeds['Seed'].apply(lambda x: int(x[1:3]))
        seeds_T1 = seeds[['Season','TeamID','seed']].copy()
        seeds_T2 = seeds[['Season','TeamID','seed']].copy()
        seeds_T1.columns = ['Season','T1_TeamID','T1_seed']
        seeds_T2.columns = ['Season','T2_TeamID','T2_seed']
        toJoin_df = pd.merge(toJoin_df, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
        toJoin_df = pd.merge(toJoin_df, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')
        toJoin_df["Seed_diff"] = toJoin_df["T1_seed"] - toJoin_df["T2_seed"]
        
        return toJoin_df


In [11]:
preprocessor = Preprocessor_NCAA(regular_results, tourney_results, seeds, True)
tourney_data = preprocessor.fit()

/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:880: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:880: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/links.py:167: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:879: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:879: RuntimeWarning: divide by

In [12]:
tourney_data.shape

(1260, 571)

In [13]:
features = [col for col in tourney_data.columns if col not in ('DayNum', 'Season', 'T1_Score', 'T1_TeamID', 'T2_Score', 'T2_TeamID') ]
print(len(features))

565


In [14]:
sub_preprocessor = Preprocessor_NCAA(regular_results,sub, seeds, False)
sub = sub_preprocessor.fit()
sub.head()

/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:880: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:880: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/links.py:167: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:879: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
/Users/s.imazeki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:879: RuntimeWarning: divide by

,ID,Pred,Season,T1_TeamID,T2_TeamID,T1_FG2ratiomean,T1_opponent_Stlmean,T1_opponent_winmean,T1_opponent_TOmean,T1_opponent_FG3ratiomean,T1_FGblockmean,T1_PFmean,T1_FG3ratiomean,T1_FGratiomean,T1_opponent_ORratiomean,T1_ORratiomean,T1_opponent_FTAmean,T1_opponent_FGAmean,T1_opponent_PFmean,T1_opponent_FGAblockmean,T1_over5PFmean,T1_opponent_DRmean,T1_opponent_FTmissedmean,T1_opponent_ORratio_opmean,T1_PF_byPeriodmean,T1_Blkmean,T1_FGmissedmean,T1_opponent_FGratiomean,T1_ASTratiomean,T1_opponent_FGmissed3mean,T1_opponent_DRratiomean,T1_opponent_FTratiomean,T1_ORmean,T1_FTmissedmean,T1_opponent_FGA3mean,T1_opponent_FTsuccess_ratiomean,T1_opponent_FG3success_ratiomean,T1_FGAblockmean,T1_opponent_FTMmean,T1_Astmean,T1_winmean,T1_FGsuccess_ratiomean,T1_opponent_DRratio_opmean,T1_FTratiomean,T1_DRmean,T1_opponent_FGMmean,T1_opponent_ASTratiomean,T1_FTMmean,T1_TOmean,T1_FGMmean,T1_Stlmean,T1_opponent_PF_byPeriodmean,T1_FTAmean,T1_FGAmean,T1_PointDiffmean,T1_DRratio_opmean,T1_FGM3mean,T1_FGA3mean,T1_opponent_FGblockmean,T1_opponent_FG2ratiomean,T1_ORratio_opmean,T1_opponent_Scoremean,T1_opponent_ORmean,T1_opponent_Astmean,T1_Scoremean,T1_FGmissed3mean,T1_FG3success_ratiomean,T1_DRratiomean,T1_opponent_FGsuccess_ratiomean,T1_Rratiomean,T1_opponent_Blkmean,T1_opponent_Rratiomean,T1_FTsuccess_ratiomean,T1_opponent_FGmissedmean,T1_opponent_FGM3mean,T2_FG2ratiomean,T2_opponent_Stlmean,T2_opponent_winmean,T2_opponent_TOmean,T2_opponent_FG3ratiomean,T2_FGblockmean,T2_PFmean,T2_FG3ratiomean,T2_FGratiomean,T2_opponent_ORratiomean,T2_ORratiomean,T2_opponent_FTAmean,T2_opponent_FGAmean,T2_opponent_PFmean,T2_opponent_FGAblockmean,T2_over5PFmean,T2_opponent_DRmean,T2_opponent_FTmissedmean,T2_opponent_ORratio_opmean,T2_PF_byPeriodmean,T2_Blkmean,T2_FGmissedmean,T2_opponent_FGratiomean,T2_ASTratiomean,T2_opponent_FGmissed3mean,T2_opponent_DRratiomean,T2_opponent_FTratiomean,T2_ORmean,T2_FTmissedmean,T2_opponent_FGA3mean,T2_opponent_FTsuccess_ratiomean,T2_opponent_FG3success_ratiomean,T2_FGAblockmean,T2_opponent_FTMmean,T2_Astmean,T2_winmean,T2_FGsuccess_ratiomean,T2_opponent_DRratio_opmean,T2_FTratiomean,T2_DRmean,T2_opponent_FGMmean,T2_opponent_ASTratiomean,T2_FTMmean,T2_TOmean,T2_FGMmean,T2_Stlmean,T2_opponent_PF_byPeriodmean,T2_FTAmean,T2_FGAmean,T2_PointDiffmean,T2_DRratio_opmean,T2_FGM3mean,T2_FGA3mean,T2_opponent_FGblockmean,T2_opponent_FG2ratiomean,T2_ORratio_opmean,T2_opponent_Scoremean,T2_opponent_ORmean,T2_opponent_Astmean,T2_Scoremean,T2_FGmissed3mean,T2_FG3success_ratiomean,T2_DRratiomean,T2_opponent_FGsuccess_ratiomean,T2_Rratiomean,T2_opponent_Blkmean,T2_opponent_Rratiomean,T2_FTsuccess_ratiomean,T2_opponent_FGmissedmean,T2_opponent_FGM3mean,T1_FG2ratio_location1mean,T1_opponent_Stl_location1mean,T1_opponent_win_location1mean,T1_opponent_TO_location1mean,T1_opponent_FG3ratio_location1mean,T1_FGblock_location1mean,T1_PF_location1mean,T1_FG3ratio_location1mean,T1_FGratio_location1mean,T1_opponent_ORratio_location1mean,T1_ORratio_location1mean,T1_opponent_FTA_location1mean,T1_opponent_FGA_location1mean,T1_opponent_PF_location1mean,T1_opponent_FGAblock_location1mean,T1_over5PF_location1mean,T1_opponent_DR_location1mean,T1_opponent_FTmissed_location1mean,T1_opponent_ORratio_op_location1mean,T1_PF_byPeriod_location1mean,T1_Blk_location1mean,T1_FGmissed_location1mean,T1_opponent_FGratio_location1mean,T1_ASTratio_location1mean,T1_opponent_FGmissed3_location1mean,T1_opponent_DRratio_location1mean,T1_opponent_FTratio_location1mean,T1_OR_location1mean,T1_FTmissed_location1mean,T1_opponent_FGA3_location1mean,T1_opponent_FTsuccess_ratio_location1mean,T1_opponent_FG3success_ratio_location1mean,T1_FGAblock_location1mean,T1_opponent_FTM_location1mean,T1_Ast_location1mean,T1_win_location1mean,T1_FGsuccess_ratio_location1mean,T1_opponent_DRratio_op_location1mean,T1_FTratio_location1mean,T1_DR_location1mean,T1_opponent_FGM_location1mean,T1_opponent_ASTratio_location1mean,T1_FTM_location1mean,T1_TO_location1mean,T1_FGM_location1mean,T1_Stl_location1mean,T1